In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import json
import re

In [2]:
# user_inf
account_opened = pd.read_csv("gotrade - account opened.csv")
kyc_started = pd.read_csv("gotrade - kyc started.csv")
kyc_completed = pd.read_csv("gotrade - kyc completed.csv")
users_data = pd.read_csv("gotrade - users data.csv")

churn_status_tracked = pd.read_csv("gotrade - churn status tracked.csv")
deposit_paid = pd.read_csv("gotrade - deposit paid.csv")

segment_event_tracked = pd.read_csv("gotrade - segment event tracked.csv")
segment_tap_event_tracked = pd.read_csv("gotrade - segment tap event tracked.csv")
session_data = pd.read_csv("gotrade - session data.csv")
trade_order_level_filled = pd.read_csv("gotrade - trade order level filled.csv")

account_opened_sorted = account_opened.sort_values(by='identifier', ascending=True)
account_opened_sorted.reset_index(drop=True, inplace=True)
kyc_completed_sorted = kyc_completed.sort_values(by='identifier', ascending=True)
kyc_completed_sorted.reset_index(drop=True, inplace=True)
kyc_started_sorted = kyc_started.sort_values(by='identifier', ascending=True)
kyc_started_sorted.reset_index(drop=True, inplace=True)
users_data_sorted = users_data.sort_values(by='identifier', ascending=True)
users_data_sorted.reset_index(drop=True, inplace=True)
# 752934
churn_status_tracked_sorted = churn_status_tracked.sort_values(by='identifier', ascending=True)
churn_status_tracked_sorted.reset_index(drop=True, inplace=True)
# 11074
deposit_paid_sorted = deposit_paid.sort_values(by='identifier', ascending=True)
deposit_paid_sorted.reset_index(drop=True, inplace=True)
# 4971560
segment_event_tracked_sorted = segment_event_tracked.sort_values(by='identifier', ascending=True)
segment_event_tracked_sorted.reset_index(drop=True, inplace=True)
# 12922536
segment_tap_event_tracked_sorted = segment_tap_event_tracked.sort_values(by='identifier', ascending=True)
segment_tap_event_tracked_sorted.reset_index(drop=True, inplace=True)
# 772058
session_data_sorted = session_data.sort_values(by='identifier', ascending=True)
session_data_sorted.reset_index(drop=True, inplace=True)
# 252496
trade_order_level_filled_sorted = trade_order_level_filled.sort_values(by='identifier', ascending=True)
trade_order_level_filled_sorted.reset_index(drop=True, inplace=True)

In [3]:
segment_event_tracked_sorted

,identifier,screen_name,event_tracked_at
0,1,ProfileScreen,2023-01-12 19:57:29.463
1,1,ProfileScreen,2022-08-16 09:25:05.78
2,1,FundsScreen,2022-09-28 12:24:59.263
3,1,BiometricPermissionScreen,2022-10-10 15:26:10.546
4,1,BiometricPermissionScreen,2022-09-28 12:24:35.35
...,...,...,...
4971555,2700,IsGotradeSafeScreen,2023-08-21 23:52:51.231
4971556,2700,IsGotradeSafeMainScreen,2023-08-21 23:52:51.23
4971557,2700,ProfileScreen,2023-08-22 00:14:58.262
4971558,2700,StockOrderSuccessScreen,2023-08-23 14:45:47.741


In [4]:
user_inf = pd.merge(account_opened_sorted, kyc_started_sorted, on='identifier', how='inner')
user_inf = pd.merge(user_inf, kyc_completed_sorted, on='identifier', how='inner')

In [5]:
column_names = ["identifier",
                "Total trading",
                "Deposit",
                "Buy",
                "Trading frequency",
                "Account setup time",
                "Churn percentage",
                "Session length"
               ]

df = pd.DataFrame(columns=column_names, dtype='int')

In [8]:
no_trading_account = []
only_buy_account = []
data_completed = datetime.strptime('2023-08-29 23:59:59', '%Y-%m-%d %H:%M:%S')
for i in range(1,2701,1):
    trade_time = []
    temp_df_trade = trade_order_level_filled_sorted[trade_order_level_filled_sorted['identifier'] == i]
    temp_df_deposit = deposit_paid_sorted[deposit_paid_sorted['identifier'] == i]
    temp_df_churn = churn_status_tracked_sorted[churn_status_tracked_sorted['identifier'] == i]
    temp_df_session = session_data_sorted[session_data_sorted['identifier'] == i]
    df_temp_event = segment_event_tracked_sorted[segment_event_tracked_sorted['identifier'] == i]
    symbols = list(set(temp_df_trade['symbol']))
    
    if len(temp_df_trade) == 0:
        no_trading_account.append(i)
        continue
    
    if len(temp_df_trade[temp_df_trade['side'] == "sell"]) == 0:
        only_buy_account.append(i)
        continue
    
    date_kyc_started_at = datetime.strptime(user_inf['kyc_started_at'][i-1], '%Y-%m-%d %H:%M:%S')
    date_account_opened_at = datetime.strptime(user_inf['account_opened_at'][i-1], '%Y-%m-%d %H:%M:%S')
    account_setup_time = abs(date_account_opened_at - date_kyc_started_at).total_seconds()/3600
    account_existing_days = abs(data_completed - date_account_opened_at).days + 1
    
    temp_df_churn_1 = temp_df_churn[temp_df_churn['is_churn'] == 1]
    
    
    Deposit_value = sum(temp_df_deposit['deposit_value_usd'])
    Buy_value = sum(temp_df_trade[temp_df_trade['side'] == 'buy']['trade_value_usd'])
    Buy_count = len(temp_df_trade[temp_df_trade['side'] == 'buy'])
    churn_percentage = len(temp_df_churn[temp_df_churn['is_churn'] == 1])/int(account_existing_days)
    
    for sub_symbol in symbols:
        max_day = max(temp_df_trade[temp_df_trade['symbol'] == sub_symbol]['trade_order_level_filled_at'])
        min_day = min(temp_df_trade[temp_df_trade['symbol'] == sub_symbol]['trade_order_level_filled_at'])

        date_max = datetime.strptime(max_day, '%Y-%m-%d %H:%M:%S')
        date_min = datetime.strptime(min_day, '%Y-%m-%d %H:%M:%S')

        difference = abs(date_max - date_min).total_seconds()/3600
        if difference == 0:
            continue
        trade_time.append(difference/len(temp_df_trade[temp_df_trade['symbol'] == sub_symbol]))
        
    Trading_frequency = np.mean(trade_time)
    session_length = sum(temp_df_session['session_length'])
    count_event = Counter(df_temp_event['screen_name'])
    if len(count_event) < 3:
        print(i)
    top_3_screen = [item[0] for item in count_event.most_common(3)]
        
    df.loc[i-1] = [int(i),
                   len(temp_df_trade),
                   round(Deposit_value,2)/len(temp_df_deposit),
                   round(Buy_value,2)/Buy_count,
                   Trading_frequency,
                   round(account_setup_time,2),
                   churn_percentage,
                   len(temp_df_session['session_length'])]

df[df.columns.tolist()[0:1]] = df[df.columns.tolist()[0:1]].astype(int)

1038
1313
1669
1900
1939
1991
2110
2127
2155
2266
2291
2313
2427
2516


In [9]:
df.reset_index(drop=True, inplace=True)
df

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length
0,1,3.0,10.320000,9.940000,55.394352,2.11,0.0,29.0
1,2,17.0,23.840000,1.969231,3432.536691,28.85,0.0,515.0
2,3,11.0,16.135000,6.416667,965.519824,26.52,0.0,195.0
3,4,10.0,17.185000,6.456000,2689.766296,25.94,0.0,363.0
4,5,8.0,13.962500,15.248000,2508.149028,5.39,0.0,479.0
...,...,...,...,...,...,...,...,...
2304,2694,19.0,4.820000,2.112000,31.598958,0.78,0.0,33.0
2305,2695,20.0,14.575000,2.235882,10.696481,0.82,0.0,48.0
2306,2696,48.0,986.048333,150.157805,11.224264,2.89,0.0,88.0
2307,2698,9.0,147.555000,109.808000,18.262014,0.61,0.0,30.0


In [10]:
df[df["identifier"] == 2110]

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length
1821,2110,28.0,8.99,5.274,51.253023,0.64,0.0,14.0


In [11]:
from pycaret.clustering import *
s = ClusteringExperiment()
s.setup(df.drop(['identifier'],axis=1), normalize = True)

,Description,Value
0,Session id,6932
1,Original data shape,"(2309, 7)"
2,Transformed data shape,"(2309, 7)"
3,Numeric features,7
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,Normalize,True
9,Normalize method,zscore


In [12]:
kmeans = s.create_model('kmeans')
s.evaluate_model(kmeans)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.4145,463.2828,1.2234,0,0,0


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [13]:
result = s.assign_model(kmeans)
result['identifier'] = df['identifier']
cols = [result.columns.tolist()[-1]] + result.columns.tolist()[:-1]
result = result[cols]
result

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length,Cluster
0,1,3.0,10.320000,9.940000,55.394352,2.110000,0.0,29.0,Cluster 0
1,2,17.0,23.840000,1.969231,3432.536621,28.850000,0.0,515.0,Cluster 0
2,3,11.0,16.135000,6.416667,965.519836,26.520000,0.0,195.0,Cluster 0
3,4,10.0,17.184999,6.456000,2689.766357,25.940001,0.0,363.0,Cluster 0
4,5,8.0,13.962500,15.248000,2508.148926,5.390000,0.0,479.0,Cluster 0
...,...,...,...,...,...,...,...,...,...
2304,2694,19.0,4.820000,2.112000,31.598959,0.780000,0.0,33.0,Cluster 0
2305,2695,20.0,14.575000,2.235882,10.696482,0.820000,0.0,48.0,Cluster 0
2306,2696,48.0,986.048340,150.157806,11.224264,2.890000,0.0,88.0,Cluster 1
2307,2698,9.0,147.554993,109.807999,18.262014,0.610000,0.0,30.0,Cluster 0


In [14]:
cluster_0 = result[result['Cluster'] == 'Cluster 0']
cluster_0

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length,Cluster
0,1,3.0,10.320000,9.940000,55.394352,2.110000,0.0,29.0,Cluster 0
1,2,17.0,23.840000,1.969231,3432.536621,28.850000,0.0,515.0,Cluster 0
2,3,11.0,16.135000,6.416667,965.519836,26.520000,0.0,195.0,Cluster 0
3,4,10.0,17.184999,6.456000,2689.766357,25.940001,0.0,363.0,Cluster 0
4,5,8.0,13.962500,15.248000,2508.148926,5.390000,0.0,479.0,Cluster 0
...,...,...,...,...,...,...,...,...,...
2303,2693,10.0,4.640000,2.887143,32.150875,1.240000,0.0,26.0,Cluster 0
2304,2694,19.0,4.820000,2.112000,31.598959,0.780000,0.0,33.0,Cluster 0
2305,2695,20.0,14.575000,2.235882,10.696482,0.820000,0.0,48.0,Cluster 0
2307,2698,9.0,147.554993,109.807999,18.262014,0.610000,0.0,30.0,Cluster 0


In [15]:
cluster_1 = result[result['Cluster'] == 'Cluster 1']
cluster_1

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length,Cluster
7,10,551.0,796.836243,973.134399,26.832960,0.460000,0.404255,531.0,Cluster 1
14,19,16.0,819.994995,627.501221,1079.768188,145.350006,0.318982,479.0,Cluster 1
29,39,43.0,936.711670,173.267303,1006.542786,7.520000,0.000000,389.0,Cluster 1
43,58,59.0,512.705994,1009.647095,319.397522,3.640000,0.000000,1366.0,Cluster 1
44,60,1193.0,1025.171997,475.504822,229.771454,15.860000,0.000000,2171.0,Cluster 1
...,...,...,...,...,...,...,...,...,...
2096,2423,42.0,663.869995,1231.849487,87.018883,296.809998,0.000000,393.0,Cluster 1
2106,2438,5.0,931.617493,679.950012,210.301880,5.960000,0.000000,70.0,Cluster 1
2109,2441,653.0,881.506653,233.428513,57.462582,0.650000,0.000000,441.0,Cluster 1
2219,2570,10.0,1109.596680,406.149994,62.867794,121.980003,0.000000,33.0,Cluster 1


In [16]:
cluster_2 = result[result['Cluster'] == 'Cluster 2']
cluster_2

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length,Cluster
12,16,8.0,86.864998,93.627502,36.212082,6476.950195,0.587045,103.0,Cluster 2
13,18,17.0,256.704987,82.127502,336.312775,3982.679932,0.304843,159.0,Cluster 2
15,20,51.0,235.630005,458.796082,104.373444,6079.299805,0.000000,1472.0,Cluster 2
18,25,41.0,40.615002,30.696400,251.718903,5976.569824,0.000000,226.0,Cluster 2
27,35,5.0,10.060000,2.502500,1589.596191,3292.060059,0.000000,104.0,Cluster 2
...,...,...,...,...,...,...,...,...,...
1695,1963,33.0,5.330000,4.571667,133.472458,2779.870117,0.000000,127.0,Cluster 2
1731,2005,14.0,36.200001,38.396252,19.914822,4147.810059,0.000000,61.0,Cluster 2
1783,2067,4.0,6.170000,4.485000,311.576599,4161.899902,0.000000,13.0,Cluster 2
1811,2098,12.0,5.010000,1.100000,0.128148,3214.540039,0.000000,12.0,Cluster 2


In [17]:
cluster_3 = result[result['Cluster'] == 'Cluster 3']
cluster_3

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length,Cluster
10,13,27.0,33.185001,6.465000,1528.316284,0.530000,0.338491,126.0,Cluster 3
11,15,20.0,10.000000,11.735714,219.453018,41.090000,0.755340,2024.0,Cluster 3
19,26,37.0,74.736664,24.010000,533.223022,96.669998,0.571150,546.0,Cluster 3
22,29,66.0,10.547500,6.348235,213.692383,37.680000,0.611650,603.0,Cluster 3
23,30,297.0,77.020714,406.908875,49.701172,3264.989990,0.595801,952.0,Cluster 3
...,...,...,...,...,...,...,...,...,...
2059,2382,331.0,523.197510,606.858887,21.540070,119.669998,0.261682,268.0,Cluster 3
2074,2398,60.0,401.372009,892.965332,7.191320,0.620000,0.317308,182.0,Cluster 3
2085,2409,2.0,50.009998,5.000000,0.052639,11.390000,0.400000,74.0,Cluster 3
2244,2601,17.0,109.625000,57.394615,204.257599,197.419998,0.371429,153.0,Cluster 3


In [18]:
cluster_0.describe()

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length
count,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000
mean,1465.413134,54.895523,66.964470,52.143627,486.382324,208.560226,0.012603,258.651337
std,762.691036,122.411133,120.398232,128.398392,696.055176,482.476105,0.047484,265.471283
min,1.000000,2.000000,4.630000,0.870000,0.009583,0.460000,0.000000,4.000000
25%,837.000000,9.000000,8.187500,4.811250,73.750717,1.575000,0.000000,80.000000
50%,1499.000000,19.000000,14.773334,10.000000,242.398438,14.860000,0.000000,170.000000
75%,2143.500000,46.000000,64.627499,39.181206,576.740295,114.239998,0.000000,332.000000
max,2700.000000,1595.000000,847.159973,1843.442261,5039.852051,2930.820068,0.328125,1600.000000


In [19]:
cluster_1.describe()

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length
count,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000
mean,1061.286624,781.038208,774.695740,1290.609619,279.005371,231.479172,0.082834,999.617859
std,720.603549,1494.997803,514.173096,3009.138184,319.264862,681.258850,0.161527,666.155457
min,10.000000,4.000000,14.910000,8.050914,0.091528,0.460000,0.000000,31.000000
25%,420.000000,71.000000,340.471039,200.052368,93.716553,0.960000,0.000000,449.000000
50%,901.000000,262.000000,776.488342,550.442383,173.127365,8.020000,0.000000,935.000000
75%,1641.000000,819.000000,1034.122070,1155.378052,319.397522,88.910004,0.059633,1393.000000
max,2696.000000,10797.000000,3007.341309,31118.480469,1820.360107,4109.200195,0.648456,3504.000000


In [20]:
cluster_2.describe()

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length
count,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000,142.000000
mean,782.211268,53.429577,81.025429,76.892769,293.827850,5368.707520,0.053116,238.056335
std,542.312566,124.613228,146.101944,199.388443,444.514984,2011.434814,0.140930,241.116394
min,16.000000,2.000000,4.690000,1.000000,0.059907,2779.870117,0.000000,12.000000
25%,351.000000,7.000000,7.390000,4.803152,64.018864,3622.569946,0.000000,78.000000
50%,714.500000,18.500000,12.000500,9.810833,160.974228,5101.250000,0.000000,172.500000
75%,1049.750000,50.000000,75.197500,46.454299,290.998779,6465.602661,0.000000,301.750000
max,2099.000000,1216.000000,703.945007,1419.979492,2437.219238,11711.990234,0.671717,1472.000000


In [21]:
cluster_3.describe()

,identifier,Total trading,Deposit,Buy,Trading frequency,Account setup time,Churn percentage,Session length
count,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000
mean,1113.788060,86.602982,84.710403,86.981773,306.281982,278.383057,0.553160,343.441803
std,702.790643,169.246964,136.345795,194.964096,351.493835,656.116699,0.163031,271.586945
min,13.000000,2.000000,5.050000,1.015135,0.007222,0.410000,0.232295,7.000000
25%,486.500000,14.000000,10.440416,8.979375,93.363914,1.880000,0.405869,153.000000
50%,1136.000000,30.000000,26.845295,18.618334,181.801910,22.709999,0.547344,286.000000
75%,1695.000000,72.500000,87.174431,69.436584,387.741058,169.354996,0.680815,456.500000
max,2648.000000,1395.000000,813.666687,1921.163818,2316.508789,4296.319824,0.875969,2024.000000
